In [8]:
import shapely
import shapely.ops
import numpy as np
import matplotlib.pyplot as plt
import fiona
from shapely.geometry import shape
import random
from shapely import Point as Point
from rtree import index
import ctypes

class CircleNode:
    def __init__(self,id,center,active_p,active_cost) -> None:
        self.id = id
        self.center = center
        self.active_p = active_p
        self.active_cost = active_cost

    def update_active(self,point,cost):
        self.active_p = point
        self.active_cost = cost

class Node:
    def __init__(self,id,pose,cost,parent = None,children = [],active=True,add_children = True):
        self.id = id
        self.pose =pose
        self.cost = cost
        self.parent = parent
        self.children = children
        self.active = active
        if parent is not None and add_children:
            parent.children.append(self)

    def updateparent(self,data):
        if self.parent is not None:
            idx = 0
            for n in self.parent.children:
                if n.id == data.id:
                    self.parent.children[idx].active = data.active
                    self.parent.updateparent(self.parent)
                    return
                idx+=1

    def updatechildren(self,data):
        if not len(self.children)==0:
            idx = 0
            for n in self.children:
                self.children[idx].parent.active = data.active
                self.children[idx].updatechildren(self.children[idx])
                idx +=1


# print(p2[0].object.active_p)
# print(p2[0].object.center)

pidx = index.Index()

PNode1 = Node(0,Point(0.0,0.0),0,None,[])
pidx.insert(PNode1.id,(PNode1.pose.x,PNode1.pose.y),id(PNode1))

PList = []
# print(PNode1.children)
for i in range(10):
    p = list(pidx.nearest((i+1,i+2),1,objects=True))
    n = ctypes.cast(p[0].object, ctypes.py_object).value
    PNodei = Node(i+1,Point(i+1,i+2),i+3,n,[])
    PList.append(PNodei)
    pidx.insert(PNodei.id,(PNodei.pose.x,PNodei.pose.y),id(PNodei))

d = list(pidx.nearest((2,1),1,objects=True))
dn = ctypes.cast(d[0].object, ctypes.py_object).value
print(dn.cost)
print(dn.pose)
print(dn.id)

dn.cost = 6
d2 = list(pidx.nearest((2,1),1,objects=True))
dn2 = ctypes.cast(d2[0].object, ctypes.py_object).value
print(dn2.cost)
print(dn2.pose)
print(dn2.id)

print(PNode1.children[0].cost)

d3 = list(pidx.nearest((20,20),1,objects=True))
dn3 = ctypes.cast(d3[0].object, ctypes.py_object).value

while not dn3.parent is None:
    print(dn3.parent.pose)
    dn3 = dn3.parent





3
POINT (1 2)
1
6
POINT (1 2)
1
6
POINT (9 10)
POINT (8 9)
POINT (7 8)
POINT (6 7)
POINT (5 6)
POINT (4 5)
POINT (3 4)
POINT (2 3)
POINT (1 2)
POINT (0 0)
